In [1]:
## Python 3.6

import os
from glob import glob
import gdal
import fiona
import ogr
from datetime import datetime
import imageio

In [2]:
import sys
sys.path.append(r'C:\Program Files\QGIS 3.0\bin')
import gdal_merge as gm

In [3]:
# Define the tif_enve_to_poly function
# Getting the extent of the DEM
# ulx, uly is the upper left corner, lrx, lry is the lower right corner
# poly is returned as an ogr geometry object/Wkt
def tif_enve_to_poly(tif_path):
    src = gdal.Open(tif_path)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)

    # Create polygon from bounding box
    # Create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(ulx, uly)
    ring.AddPoint(ulx, lry)
    ring.AddPoint(lrx,lry)
    ring.AddPoint(lrx, uly)
    ring.AddPoint(ulx, uly)

    # Create polygon
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    
    return poly

In [4]:
# Paths
txt_file = r'C:\Users\Administrator\Data\HurricaneMichael\intersects_lists.txt'
extent_shpfile = r'C:\Users\Administrator\Data\HurricaneMichael\Mexico_Beach.shp'
tiff_dir = r'C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB'

tiff_filetype = '.tif'

In [5]:
tiffs = glob(os.path.join(tiff_dir, '*{}'.format(tiff_filetype)))
print(len(tiffs))

534


In [6]:
# Start your engines
start = datetime.now()
print('Start time = ', start, '\n')

driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(extent_shpfile, 0)
layer = dataSource.GetLayer()

tiff_intersect_list = []
tiff_intersect_paths = []

for feature in layer:
    MEgeom = feature.GetGeometryRef()

    for i, tiff in enumerate(tiffs):
        tiffname = os.path.basename(tiff).split('.')[0]
        tiff_extent = tif_enve_to_poly(tiff)
        if i %100 == 0:
            print(i, '/', len(tiffs))
        
        if tiff_extent.Intersect(MEgeom):
            print(tiff)
            tiff_intersect_list.append(tiffname)
            tiff_intersect_paths.append(tiff)

with open(txt_file, 'w') as text:
    text.write('There are {} tiffs that intersect.\n\nTiff names:\n'.format(len(tiff_intersect_list))\
               + str(tiff_intersect_list) + '\n\nTiff Paths:\n' + str(tiff_intersect_paths))
    
# Game over
print('Time complete: ', datetime.now())
proc_time = datetime.now()-start
print('Processing time = ', proc_time)

Start time =  2018-10-12 23:13:41.433430 

0 / 534
100 / 534
C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852530w295700n.tif
C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852530w295830n.tif
C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852700w295700n.tif
C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852700w295830n.tif
200 / 534
300 / 534
400 / 534
500 / 534
Time complete:  2018-10-12 23:13:55.892373
Processing time =  0:00:14.460944


In [7]:
# merge the post hurricane imagery
# lazy hard code
gm.main(['', '-o', r'C:\Users\Administrator\Data\HurricaneMichael\mex_beach_post.tif',\
         r'C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852530w295700n.tif',\
         r'C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852530w295830n.tif',\
         r'C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852700w295700n.tif',\
         r'C:\Users\Administrator\Data\HurricaneMichael\20181011a_RGB\20181011aC0852700w295830n.tif']) 

In [8]:
# merge the pre hurricane imagery
# lazy hard code
gm.main(['', '-o', r'C:\Users\Administrator\Data\HurricaneMichael\mex_beach_pre.tif',\
         r'C:\Users\Administrator\Data\HurricaneMichael\USGS\High Resolution Orthoimagery\2988950_67125\FL\2012\201212_bay_county_fl_10in_sp_clr\vol001\67125.tif',\
         r'C:\Users\Administrator\Data\HurricaneMichael\USGS\High Resolution Orthoimagery\2989445_67126\FL\2012\201212_bay_county_fl_10in_sp_clr\vol002\67126.tif'])

In [ ]:
### use QGIS to visually clip and project the imagery
### MAKE SURE TO SET RESOLUTION TO BE THE SAME FOR BOTH IMAGES

In [9]:
# Make the gif, ignore the fact that imageio hates my guts
# lazy hard code
filenames = [r'C:\Users\Administrator\Data\HurricaneMichael\gif_images\pre_small.tif', r'C:\Users\Administrator\Data\HurricaneMichael\gif_images\post_small.tif']
exportname = r'C:\Users\Administrator\Data\HurricaneMichael\gif_images\H_M_final.gif'

images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(exportname, images, format='GIF', duration=5)

C:\Users\Administrator\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_packbits'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\Administrator\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_lzw'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\Administrator\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'unpack_ints'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\Administrator\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'reverse_bitorder'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
